In [2]:
import random

class EllipticCurve:
    def __init__(self, a, b, q):
        self.a = a
        self.b = b
        self.q = q

    def is_point_on_curve(self, x, y):
        return (y ** 2 - (x ** 3 + self.a * x + self.b)) % self.q == 0

class Point:
    def __init__(self, curve, x, y):
        self.curve = curve
        self.x = x
        self.y = y

    def __add__(self, other):
        if self.curve != other.curve:
            raise ValueError("Points are not on the same curve")
        if self.x is None:
            return other
        if other.x is None:
            return self
        if self.x == other.x and self.y != other.y:
            return None
        if self == other:
            lam = (3 * self.x ** 2 + self.curve.a) * pow(2 * self.y, -1, self.curve.q)
        else:
            lam = (other.y - self.y) * pow(other.x - self.x, -1, self.curve.q)
        x3 = (lam ** 2 - self.x - other.x) % self.curve.q
        y3 = (lam * (self.x - x3) - self.y) % self.curve.q
        return Point(self.curve, x3, y3)

    def __mul__(self, scalar):
        res = None
        for i in range(scalar.bit_length()):
            if (scalar >> i) & 1:
                res += self * (1 << i)
        return res

def generate_key_pair(curve, generator):
    private_key = random.randint(1, curve.q - 1)
    public_key = generator * private_key
    return private_key, public_key

def generate_system_parameters():
    q = 2**256 - 2**32 - 977

    # Choose a and b in Fq where 4a^3 + 27b^2 ≠ 0
    a = random.randint(0, q - 1)
    b = random.randint(0, q - 1)
    while (4 * a**3 + 27 * b**2) % q == 0:
        a = random.randint(0, q - 1)
        b = random.randint(0, q - 1)

    # Choose a point P on the curve
    curve = EllipticCurve(a, b, q)
    while True:
        x = random.randint(0, q - 1)
        y = random.randint(0, q - 1)
        if curve.is_point_on_curve(x, y):
            P = Point(curve, x, y)
            break

    return q, P, curve

# Initializing phase
def initialize_phase(num_PNs):
    # Generate system parameters
    q, generator, curve = generate_system_parameters()

    
    phi = [random.randint(1, q - 1) for _ in range(num_PNs)]
    phi_0 = 1
    for i in range(num_PNs):
        phi_0 *= phi[i]
    phi_0 %= q

 
    def phi_polynomial(x):
        res = phi_0
        for i in range(num_PNs):
            res += phi[i] * pow(x, i + 1, q)
        return res % q

    # Publicize system parameters
    public_parameters = {'q': q, 'generator': generator, 'curve': curve, 'phi_polynomial': phi_polynomial}
    print("Public system parameters:", public_parameters)


initialize_phase(5)  # Pass the number of PNs in the network


KeyboardInterrupt: 